# make an AOI detector/drawer in h3
"""
1. ETL - polars
    convert datetime from string to epoch
    convert lat lon to h3
        resolution - 
    
2. Enrichment
    POI table - 
        decide who is of interest, maybe select some sketchy ports and use those to tag
        can we get vessel nationalitys? is there a vessel registration API????
        
3. Scoring - RFM style
    to dupe redline - port polygons - World Port Index
        https://msi.nga.mil/Publications/WPI     ### THIS HAS THE FILE USED BELOW
        
        POLYGONS / shape files - REST API
            https://www.marineregions.org/downloads.php
            
        https://data.humdata.org/dataset/global-ports
        https://hub.arcgis.com/datasets/EDT::world-port-index/explore?location=0.017789%2C1.000000%2C1.88
        https://datacatalog.worldbank.org/search/dataset/0060145
        
        portsheds?
            https://datacatalog.worldbank.org/search/dataset/0060145
        
        
        open street map?
        

3. Counts for naive bayes
        total per hex
        POI per hex
        
4. Cluster hex's

5. Merge cluster groups into polygons

6. Match polygons to ports

7. Make list of 'nefarious' ports
    label intersecting polygons
    
8. use scan_csv to make hex probability assignments
    use logging to retain calculations from previous files so new #'s are updates

"""



In [1]:
# !pip install folium geohash 

In [8]:
import pandas as pd
import polars as pl
"""
https://pola-rs.github.io/polars/py-polars/html/reference/expressions/index.html
https://pola-rs.github.io/polars-book/user-guide/introduction.html
"""

import geopandas as gpd
# import geopolars as gpl
"""
https://geopolars.org/
"""

import folium

import shapely
# import geohash

from shapely.geometry import Polygon, Point
import shapely.wkt

import os

import math
import h3

In [13]:
# set some globals
TESTING = False
ETL_PATH = r"'/mnt/d/data/AIS/ais_csv/h3_processed"

In [14]:
os.getcwd()

'/mnt/c/Users/Asus/Code/parliament/notebooks'

In [16]:
# Port files breakdown
port = pd.read_csv('/mnt/d/data/Ports/UpdatedPub150.csv')
port = pl.from_pandas(port)

# drop most the columns. could be interesting for things though....
port = port[['World Port Index Number', 'Region Name', 
      'Main Port Name', 'Alternate Port Name', 
      'UN/LOCODE', 'Country Code', 'World Water Body', 
#       'IHO S-130 Sea Area', 'Sailing Direction or Publication', 
#       'Publication Link', 'Standard Nautical Chart', 
#       'IHO S-57 Electronic Navigational Chart', 
#       'IHO S-101 Electronic Navigational Chart', 
#       'Digital Nautical Chart', 'Tidal Range (m)', 
#       'Entrance Width (m)', 'Channel Depth (m)', 
#       'Anchorage Depth (m)', 'Cargo Pier Depth (m)', 
#       'Oil Terminal Depth (m)', 'Liquified Natural Gas Terminal Depth (m)', 
      'Maximum Vessel Length (m)', 'Maximum Vessel Beam (m)', 
      'Maximum Vessel Draft (m)', 'Offshore Maximum Vessel Length (m)', 
      'Offshore Maximum Vessel Beam (m)', 'Offshore Maximum Vessel Draft (m)', 
      'Harbor Size', 'Harbor Type', 'Harbor Use', 
#       'Shelter Afforded', 
#       'Entrance Restriction - Tide', 'Entrance Restriction - Heavy Swell', 
#       'Entrance Restriction - Ice', 'Entrance Restriction - Other', 
#       'Overhead Limits', 'Underkeel Clearance Management System', 
      'Good Holding Ground', 'Turning Area', 
      'Port Security', 
#       'Estimated Time of Arrival Message', 'Quarantine - Pratique', 
#       'Quarantine - Sanitation', 'Quarantine - Other', 
#       'Traffic Separation Scheme', 'Vessel Traffic Service', 
      'First Port of Entry', 
#       'US Representative', 'Pilotage - Compulsory', 
#       'Pilotage - Available', 'Pilotage - Local Assistance', 'Pilotage - Advisable', 
#       'Tugs - Salvage', 'Tugs - Assistance', 'Communications - Telephone', 
#       'Communications - Telefax', 'Communications - Radio', 
#       'Communications - Radiotelephone', 'Communications - Airport', 
#       'Communications - Rail', 'Search and Rescue', 'NAVAREA', 
#       'Facilities - Wharves', 'Facilities - Anchorage', 
#       'Facilities - Dangerous Cargo Anchorage', 'Facilities - Med Mooring', 
#       'Facilities - Beach Mooring', 'Facilities - Ice Mooring', 
#       'Facilities - Ro-Ro', 'Facilities - Solid Bulk', 'Facilities - Liquid Bulk', 
#       'Facilities - Container', 'Facilities - Breakbulk', 
#       'Facilities - Oil Terminal', 'Facilities - LNG Terminal', 
#       'Facilities - Other', 'Medical Facilities', 'Garbage Disposal', 
#       'Chemical Holding Tank Disposal', 'Degaussing', 
#       'Dirty Ballast Disposal', 'Cranes - Fixed', 
#       'Cranes - Mobile', 'Cranes - Floating', 
#       'Cranes - Container', 'Lifts - 100+ Tons', 
#       'Lifts - 50-100 Tons', 'Lifts - 25-49 Tons', 
#       'Lifts - 0-24 Tons', 'Services - Longshoremen', 
#       'Services - Electricity', 'Services - Steam', 
#       'Services - Navigation Equipment', 'Services - Electrical Repair', 
#       'Services - Ice Breaking', 'Services - Diving', 
#       'Supplies - Provisions', 'Supplies - Potable Water', 
#       'Supplies - Fuel Oil', 'Supplies - Diesel Oil', 'Supplies - Aviation Fuel', 
#       'Supplies - Deck', 'Supplies - Engine', 
      'Repairs', 
      'Dry Dock', 'Railway', 
      'Latitude', 'Longitude']]

# sorted(port['Country Code'].unique())

In [21]:
# portsheds???? what are these?
df = pd.read_csv('/mnt/d/data/Ports/major_ports.csv')
df

,OBJECTID,Postal,Lat,Lng,Type,ISO3,REGION
0,12,"Mongla (Chalna),Bangladesh",22.488900,89.595800,Bulk,BGD,South Asia
1,19,"Suape (Recife),Brazil",-8.055556,-34.891111,General,BRA,Latin America & Caribbean
2,20,"PecΘm (near Fortaleza),Brazil",-3.533611,-38.785833,General,BRA,Latin America & Caribbean
3,23,"Ponta da Madeira,Brazil",-2.565000,-44.370000,Bulk,BRA,Latin America & Caribbean
4,24,"Ponta do Ubu,Brazil",-20.788500,-40.573700,Bulk,BRA,Latin America & Caribbean
...,...,...,...,...,...,...,...
172,134,"Surigao,Philippines",9.789000,125.495000,Bulk,PHL,East Asia & Pacific
173,130,"Davao,Philippines",7.073060,125.612780,General,PHL,East Asia & Pacific
174,121,"Port Moresby ,Papua New Guinea",-9.477230,147.150890,General,PNG,East Asia & Pacific
175,176,Lamu Mainland,NaN,NaN,General,KEN,Sub-Saharan Africa


In [22]:
"""
about h3
    https://www.uber.com/blog/h3/
    
resolutions... 122 hex's in base layer, each layer down is 1/7th the area per, 16 layers
sphere SA=4πr^2
radius earth = 6378.1 kilometers
"""

# this finds hex area by resolution level
re = 6378100
sae = 4*math.pi *(re**2)

print("Resolution 1\t", round(sae), 'm')

for a in range(1,16):
    sae = sae/7
    print(f"Resolution {a+1}\t", round(sae), 'm')

Resolution 1	 511201962310545 m
Resolution 2	 73028851758649 m
Resolution 3	 10432693108378 m
Resolution 4	 1490384729768 m
Resolution 5	 212912104253 m
Resolution 6	 30416014893 m
Resolution 7	 4345144985 m
Resolution 8	 620734998 m
Resolution 9	 88676428 m
Resolution 10	 12668061 m
Resolution 11	 1809723 m
Resolution 12	 258532 m
Resolution 13	 36933 m
Resolution 14	 5276 m
Resolution 15	 754 m
Resolution 16	 108 m


# AIS in H3 Analysis

## AIS
### its what we have, i use some csvs i had but i think the api is open
 
"""
what is AIS? looks like some interesting papers @ the end
    https://en.wikipedia.org/wiki/Automatic_identification_system
    
free sources

    EXCELLENT article - plys how to build a receiver. I like this
    https://xyzt.ai/2020/07/09/where-to-obtain-ais-data-and-easily-analyze-it-without-writing-a-single-line-of-code/#:~:text=A%20great%20source%20of%20free,published%20on%20an%20FTP%20site.
    
    global api
    https://aisstream.io/
    
    us waters only in csv's
    https://marinecadastre.gov/ais/
    
    norwegian api
    https://kystverket.no/navigasjonstjenester/ais/tilgang-pa-ais-data/
    
paid sources (free trials)
    https://www.marinetraffic.com/en/ais-api-services


discussion of the columns - different source though
    https://api.vtexplorer.com/docs/response-ais.html
    
mmsi / id lookup
    https://wireless2.fcc.gov/UlsApp/UlsSearch/searchShip.jsp
"""

In [25]:
# # free global API
# # https://aisstream.io/documentation
# # my api key  - 
# apikey = 'b4871d51295ea8305148164f169d8536bc302cdf'
# # https://leonrichter.de/posts/pyais/
# # https://pypi.org/project/libais/

# # from setuptools import setup
# # setup(
# #     name='aisstream.io-example-simple',
# #     author='aisstream.io',
# #     version='0.1',
# #     install_requires=[
# #         'websockets',
# #         'asyncio'
# #     ],
# # )
# import asyncio
# import websockets
# import json
# from datetime import datetime, timezone

# def async def connect_ais_stream():

#     async with websockets.connect("wss://stream.aisstream.io/v0/stream") as websocket:
#         subscribe_message = {"APIKey": "<YOUR API KEY>", "BoundingBoxes": [[[-180, -90], [180, 90]]]}

#         subscribe_message_json = json.dumps(subscribe_message)
#         await websocket.send(subscribe_message_json)

#         async for message_json in websocket:
#             message = json.loads(message_json)
#             message_type = message["MessageType"]

#             if message_type == "PositionReport":
#                 # the message parameter contains a key of the message type which contains the message itself
#                 ais_message = message['Message']['PositionReport']
#                 print(f"[{datetime.now(timezone.utc)}] ShipId: {ais_message['UserID']} Latitude: {ais_message['Latitude']} Longitude: {ais_message['Longitude']}")

# # if __name__ == "__main__":
# asyncio.run(connect_ais_stream())
    
os.listdir('/mnt/d/data/AIS')

['.ipynb_checkpoints',
 'ais_csv',
 'bb_ec50edf2-e301-4dc8-8422-af0425265dfd.csv',
 'bb_f77084a2-b037-4d18-ba4c-f85f0120f99a.csv',
 'data_webmercator_coords',
 'From where',
 'global AIS from NOAA',
 'NOAA FTP download',
 'spatial temporal clustering AIS with BB.ipynb',
 'Untitled.ipynb']

In [26]:
ais_path = '/mnt/d/data/AIS/ais_csv'
files = [i for i in os.listdir(ais_path) if '.csv' in i]

# read in a whole day/file
print(files[-1])
pf = pl.read_csv(ais_path+'\\'+files[-1])



# datetime doesnt warrent a function
# https://pola-rs.github.io/polars/py-polars/html/reference/series/timeseries.html
pf = pf.with_column(pl.col('BaseDateTime').str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S"))
pf.head()

AIS_2021_03_28.csv


FileNotFoundError: No such file or directory (os error 2): /mnt/d/data/AIS/ais_csv\AIS_2021_03_28.csv

In [ ]:
# add H3 feature and put timestamps in datetime format
def pl_h3(pf, lat_col='LAT', lon_col='LON', new_col='H3', resolution=16):
    """
    ### TODO: write a preprocessor function that uses this and re-saves the files
    
    this converts lat lon to h3 in polars
    (function to apply a function to 2 columns in polars....)
    """
    return pf.with_columns(pl.struct([lon_col,lat_col]).apply(lambda x: h3.geo_to_h3(lat=x[lat_col], lng=x[lon_col], resolution=7)).alias(new_col))


pf = pl_h3(pf, lat_col='LAT', lon_col='LON', new_col='H3', resolution=16)
pf.head()

In [ ]:
# add H3 feature to ports - resolution of 10 chosen doing visual comparison with vessels that seem to have gone there
port = pl_h3(port, lat_col='Latitude', lon_col='Longitude', new_col='H3', resolution=8)

### These borrowed plotting functions do clusters and dont account for hex resolution..... fix it

In [ ]:
# plotting modified from https://github.com/uber/h3-py-notebooks/blob/master/notebooks/usage.ipynb
zoom = 10
def visualize_hexagons(hexagons, color="red", folium_map=None):
    """
    hexagons is a list of hexcluster. Each hexcluster is a list of hexagons. 
    eg. [[hex1, hex2], [hex3, hex4]]
    """
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)
    
    if folium_map is None:
        m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=zoom, tiles='cartodbpositron')
    else:
        m = folium_map
    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
        m.add_child(my_PolyLine)
    return m
    

def visualize_polygon(polyline, color):
    polyline.append(polyline[0])
    lat = [p[0] for p in polyline]
    lng = [p[1] for p in polyline]
    m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=zoom, tiles='cartodbpositron')
    my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
    m.add_child(my_PolyLine)
    return m

In [ ]:
# pick an individual vessel to start with a case study and write out the functionality
mmsi = 316003664
pf['MMSI'].value_counts().head()

In [ ]:
# h3_address = h3.geo_to_h3(37.3615593, -122.0553238, 9) # lat, lng, hex resolution
vessel = pf.filter(pl.col('MMSI') == mmsi)
vessel.head()

### Look at a single vessel

In [ ]:
m = visualize_hexagons(vessel['H3'].unique())
display(m)

### I think if you fit a curve through the centroids youd find the path.... or just plot the lat/lon's

In [ ]:
# this time around we dont want to use python dictrionaries to store!!!
# to make it faster we are going to use polars datastructures to count and calc by hex bin

# pandas value counts equivalent
vessel.select(pl.col('H3')).n_unique()

### All the ports

In [ ]:
m = visualize_hexagons(port['H3'].unique())
display(m)

### counts per hex for a single vessel

In [ ]:
q = (
    vessel.lazy()
    .groupby("H3")
    .agg(
        [
            pl.max('BaseDateTime'),
            pl.count(),
            pl.n_unique('BaseDateTime').alias("BaseDateTime_count"),
            
        ]
    )
    .sort("count", reverse=True)
)

q = q.collect()
q.head(15)

### What ports was the vessl seen at????

In [ ]:
visited = pl.DataFrame(port['H3']).join(pl.DataFrame(vessel['H3']),
                                        on="H3",
                                        how='inner').unique()
visited

## Based on Visual Inspection....

### I think we should be making the vessel intersection be within 1 hex, rather than the exact hex

If we do so then the result becomes as follows..

In [ ]:
# add neighbors to ports - easier - name them same as port - or harder - do an aggregation
buff_ports = port['Main Port Name',  'H3', 'Country Code', 'World Water Body', 'Longitude', 'Latitude']



len('872a1061bffffff')

In [ ]:
gridDistance = 2

#test
cell = buff_ports['H3'][0]
print('Hex in:\n\n', cell, '\n\nNeighbors out:')
# h3.hex_ring(cell, gridDistance) # unordered set of cells with H3 distance, so 1 == one cell away
h3.k_ring(cell, gridDistance) # k_ring is allegedly faster but slightly less accurate, but i like that it includes the originial cell....

In [ ]:
# apply by row (selection context) - find all neighbors, unpack neighbors and give same label....
neigbor_ports = buff_ports[ 'H3', 
                           'Main Port Name', 
                           'Country Code',
                          ].with_columns(pl.col("H3").apply(lambda cell: list(h3.k_ring(cell, 
                                                                                        gridDistance))).alias("h3")).explode('h3')
neigbor_ports.head()

In [ ]:
# find new intersections with the vessel using neighbors - h3 column
neighbors_visited = neigbor_ports.join(pl.DataFrame(vessel['H3']),
                                       left_on='h3',
                                       right_on="H3",
                                       how='inner').unique()
neighbors_visited['Main Port Name'].unique()

###  Now for RFM......
https://www.geeksforgeeks.org/rfm-analysis-analysis-using-python/

Select our ports

Scan files to get rfm scores by vessel

In [ ]:
# designate ports
aois = port.filter(pl.col("H3").is_in(neighbors_visited["H3"].to_list()))['Main Port Name'].to_list()
aois

### This reads through every file. pick and chose what to keep

In [ ]:
# # will probably be better to scan the csv's in the long run so we can handle all the data.... here is how
# results = {}

# to_read = 2 # number of files to read, -1 goes to the end

# # this gets all the data for a vessel from all files
# for i in range(len(files[:to_read])):
#     s = (pl.scan_csv(ais_path+'\\'+files[i])
#         .filter(pl.col("MMSI") == 316044349) # select all in the file with matching mmsi
#         .groupby("BaseDateTime") # gets only one obs per value in this column (timestamp basically returns all), to use H3 we have to resave all the files....
#         .agg(pl.all().sum())
#         ) # and this would find one instance of every hex it was in, if we groupby H3
#     results[files[i]] = s.collect()


### Get ALL the vessel id's to use while we get scores (so we know whats missing)

In [ ]:
# test getting all the vessel numbers out of the files
mmsi = pl.concat(
    [pl.scan_csv(ais_path+'\\'+files[i]).select(pl.col("MMSI").unique()).collect() for i in range(len(files))]
)
mmsi = mmsi['MMSI'].unique()

In [ ]:
"""
so should we scan by....
                        port
                        hex
                        hex cluster (for plotting)
                            which is basically port neighbors
                        vessel
                    file - well we HAVE to scan by file, i suppose
                    
depends on the task.... each if better for something different, but we dont want to scan multiple times....
        or do we???
        by file is forced so lets do them all on the working query
        gonna need logging, 
            we have to remember what we did and how much went into so we can adjust some stats as we go
"""

In [ ]:
from datetime import datetime, timedelta

start_time = datetime.now()
start_time = start_time-timedelta(minutes=12) # test
start_time-datetime.now()

In [ ]:
from IPython.display import clear_output
completed
outfile

### ETL AIS files with H3 encoding into parquet files

In [ ]:
# this gets the base data we are using per vessel, from each individual file
start_time = datetime.now()
results={}
to_read = -1

tot = len(files)
completed = os.listdir(ETL_PATH)

for i in range(len(files[:to_read])):
    
    clear_output(wait=True)
    print('Performing H3 encoding and parquet encoding')
    print(f"\tProcessing file {i+1}/{tot}:\t {files[i]}")
    
    
    filename = files[i].replace('.csv', '_h3.parquet')    
    outfile = f"{ETL_PATH}{os.sep}{filename}"
    
    if filename in completed:
        continue
    
    s = pl.scan_csv(ais_path+'\\'+files[i]).select(["MMSI", "BaseDateTime", "LAT", "LON"])
    
    # encode h3.....
    s = pl_h3(s, lat_col='LAT', lon_col='LON', new_col='H3', resolution=16)
    
    results[files[i]] = s.groupby("MMSI").agg(
        [
            pl.max('BaseDateTime').alias('last_seen'),
            pl.col('BaseDateTime'),
            pl.col('H3'),
            pl.col("LAT"),
            pl.col("LON")
        ])
    
    # too big still, so instead we will save these as files and scan them 
    # since we are re-saving, we might as well switch to parquet for efficiency


    s.collect().write_parquet(outfile)
    
    if TESTING == True:
        break
completed = os.listdir(ETL_PATH)
# i think this is still too big once we get ALL the files
# results = pl.concat(list(results.values())).collect()

# calculate elapsed time
elapsed = start_time-datetime.now()
elapsed = round(abs(elapsed.total_seconds()),1) # put into minutes at some point

print('___________________________________________________\nOperation took:', elapsed, 'seconds')

In [ ]:
# now we can filter by mmsi and use some kind of explode to look at the lists of H3 encodings
filename = f"{ETL_PATH}{os.sep}{completed[0]}"
pl.read_parquet(filename)

In [ ]:
pq = pl.scan_parquet(filename).collect()

In [ ]:
pq.groupby('MMSI').head()

In [ ]:
s.groupby("MMSI").agg(
    [
        pl.max('BaseDateTime').alias('last_seen'),
]).collect()

#### do RFM calculations by vessel with groupby

In [ ]:
"""
should make a historic score for these..... back propogate(not what i mean) and track - poi table

recency can have an opposite, anti recency, or mean etc. rarity maybe (this is an odd port to visit)
    maybe match this stuff up with news reporting?????
    
frequency
    how often they are visiting
    match with news reporting???
"""

recent_date = pf['BaseDateTime'].max()

q = (
    pf.lazy()
    .groupby("MMSI")
    .agg(
        [
            pl.col('H3').unique(),
            
            # figuring out how to apply a function to a column in a group.... almost there. Problem is | acc=pl.lit() | i think
            # i suppose we could just do it afterward.................
#             pl.fold(acc=pl.lit(0),
#                     f=lambda acc, x: acc.total_seconds(),
#                     exprs=(pl.max('BaseDateTime')-recent_date)
#                    ).alias('recency_score'),
            
            
            #recency
            pl.max('BaseDateTime').alias('last_seen'), # recency
            (pl.max('BaseDateTime')-recent_date).alias('recency_score'), # this gives the score but needs to be done across days.... this file is all today only
            # agism
            pl.min('BaseDateTime').alias('first_seen'), # anti-recency / age
            # score
            
            # frequency - count of times at port.....
            pl.count().alias('observations'),
            # score
            
            
            # std in these is a measure of how far they typically travel - within this file anyway
#             pl.std('LAT').alias('lat_std'),
#             pl.std('LON').alias('lon_std'),
            
            # M, value - need an equation for this, maybe combo of home port, country of registration, # other aois visited
            
            
            # gather vessel name, class etc. more than one means changes. why change??? youre suspect
#             pl.col('CallSign').n_unique(),
#             pl.col('VesselName').n_unique(),
#             pl.col('VesselType').n_unique(),
#             pl.col('TranscieverClass').n_unique(),
#             pl.col('IMO').n_unique(),
#             pl.col('Length').n_unique(),
#             pl.col('Width').n_unique(),
#             pl.col('Draft').n_unique(),
            
            
            # get all the values theyve been seen as - look them up, maybe a pattern? or many is weird? idk
#             pl.col('Status').unique(),
#             pl.col('Cargo').unique(), # we may be able to match this against ports as those list product types
            # ie, are you bringing the normal thing? how many things do you transport? just one?
        ]
    )
).sort("recency_score", reverse=False)

j = q.collect()
j

## Scan files to find counts by hex by timeperiod

### parameters
hex resolution - 
timeperiod - try day and hour

## Scan files to find PORT VISITS by hex by timeperiod

### parameters

### get vessel port path

## using DAG's? or other graph techniques

### nefarious paths - combinations....

not just one to one, here to there

### find path combinations from origin x to destination y

ie. going from tehran to crimea given some start and end windows, what paths exist?

parameters
    start and end window sizes
    load/unload window time

### any graph based AI/ML algs that can help us?

In [ ]:
# open street map and other location info
"""

https://date.nager.at/

"""

In [ ]:
# gpsd - drones, android map service, computers, robot submarines, and driverless cars.
# ncreasingly common in recent generations of manned aircraft, marine navigation systems, and military vehicles
"""
https://gpsd.gitlab.io/gpsd/index.html
"""

In [ ]:
# flight tracker data
"""
free
    https://opensky-network.org/

some free
    https://aviation-edge.com/
"""

In [ ]:
# wifi data
"""
some free
    https://wigle.net/
"""

In [ ]:
# news
"""
https://openbb.co/

holidays
    https://date.nager.at/
    

"""

In [ ]:
# weather
"""
https://www.7timer.info/doc.php?lang=en
"""

In [ ]:
# social media api's - can we strip locations from somewhere? like tiktok....


In [ ]:
# todo... build a couple libraries to hit the above APIs how we would like to
